In [4]:
from LIPA2.ilaplace import *
from LIPA2.qp_solver import *

from utils import *
from utils.interpolate_1D import *


import numpy as np
norm=np.linalg.norm

syms_def(locals(),' c0, c1 ,c0,c1,c2,c3,c4,b1, b2, b3, b4, g, a1, a2, a3, a4, g1')

tic();

import dill
dill.settings['recurse'] = True

fname='ilapi0.ipynb.picle'
if 0:
    #[p,d,q]=sym_reduce([g],[( [b1,b2,b3,b4] ,g  )]);
    [p,d,q]=sym_reduce([c0,c1,c2,c3],[( [b1,b2,b3,b4] ,g  )]);
    dill.dump([p,d,q],open(fname,'wb'))
else:
    [p,d,q]=dill.load(open(fname,'rb'))
    
#[p,d,q]=sym_reduce([c0 ,c1,c0,c1,c2,c3],[( [0,b1,b2,b3] ,g  )]);

toc('reduce:')
#jo=arg2jso(c1=1.1,c2=2.3,c3=1.11,c4=21,c0=10j,g=3.11,g1=2j,b1=2.1,b2=1.2,b3=1.1j,a4=2)
jo={
    
    c0:-5.5+250j,
    c1:-5.5-250.5j,
    c2:-1.5+253j,
    c3:-1.5-253j,
    c4:-21,
    
    g:-17.5+250j,
    #g:0,                
    b1:0,
    b2:-1j,
    b3:0,
    b4:1,
    }



tic();
nd=0
nnd=5
'''
[GF,exg]=ilaplace_functor(p,jo,nd)
[F,exf]=ilaplace_functor(q,jo)
'''

jetGF=ilaplace_jet(p,jo)
jetF=ilaplace_jet(q,jo)

GF=jetGF(range(nnd))
F=jetF(range(nnd))

toc('functors:')



reduce: 0.0177883 sec
functors: 2.9081 sec


2.9081033999999875

In [5]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib widget
#fig=plt.figure(figsize=(12,8))
fig, ax = plt.subplots(figsize=(12,8))
#fig.suptitle('sensors data + pure response ', fontsize=16)
ax.grid(True,which='major')
ax.minorticks_on()
ax.grid(True,which='minor',alpha=0.2)
tt=np.linspace(0,10,7200)
rn=lambda x: x.real/np.max(np.abs(x.real))
#plt.plot(tt,rn(F(tt)),tt,rn(GF(tt)))

nd=0
ax.plot(tt,rn(F(tt)[nd]),label='F')
ax.plot(tt,rn(GF(tt)[nd]),label='G');
legend = ax.legend(loc='upper right', shadow=True, fontsize='x-large')
''


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

''

In [6]:
[DC,QP,G,FF]=make_lipa_data(d,q,jo);

LM=[6,8]
LM=[10,12]

dt=0.0225*1.8

'''
print(QP.shape)
print(G.shape)
print(FF.shape)
'''
lqp=lipa_qp_number(DC,LM=LM,FF=FF,qp=QP,g=G,nd=8).reset(dt);
nd=0
nt=100
tt=dt*np.arange(1,nt+1)
tm=np.max(tt)
ttx=np.linspace(0,tm,10000)
x=GF(tt)[nd]
ynn=lqp.dump(nt)
ynn=ynn.reshape(ynn.shape[0],-1).T
y=ynn[nd]
deg=ynn.shape[0]-1

jG=jet_spline(tt,ynn,deg=7)

err=norm(x-y)/norm(x);
#fig2=plt.figure(figsize=(12,8))
fig2, ax2 = plt.subplots(figsize=(12,8))

if 1:
    ax2.plot(ttx,GF(ttx)[nd].real,label='exact')
    ax2.plot(ttx,jG(ttx).real,label='spline')
    ax2.plot(tt,y.real, marker='o',linewidth=0,label='step point')
else:
    ax2.plot(tt,y.real, marker='o',linewidth=0,label='step point')
legend = ax2.legend(loc='upper right', shadow=True, fontsize='x-large')
#print(x)    
#print(y[0])    
s=sprintf('error=%3.4g%%\n',100*err)

ax2.set_title(s)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'error=0.01723%\n')

In [15]:
deg

7

In [ ]:
plt.plot(tt,y, marker='o',linewidth=0)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(tt,y.real, marker='o',linewidth=0,label='ss')
#legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')


In [5]:
s=jetGF.getex();s


(b1/((-c0 + c1)*(c1 - c2)*(c1 - c3)*(c1 - g)) + b2/((-c0 + c1)*(c1 - c2)*(c1 - c3)*(c1 - g)**2) + b3/((-c0 + c1)*(c1 - c2)*(c1 - c3)*(c1 - g)**3) + b4/((-c0 + c1)*(c1 - c2)*(c1 - c3)*(c1 - g)**4))*exp(c1*t) + (b1/((-c0 + c2)*(-c1 + c2)*(c2 - c3)*(c2 - g)) + b2/((-c0 + c2)*(-c1 + c2)*(c2 - c3)*(c2 - g)**2) + b3/((-c0 + c2)*(-c1 + c2)*(c2 - c3)*(c2 - g)**3) + b4/((-c0 + c2)*(-c1 + c2)*(c2 - c3)*(c2 - g)**4))*exp(c2*t) + (b1/((-c0 + c3)*(-c1 + c3)*(-c2 + c3)*(c3 - g)) + b2/((-c0 + c3)*(-c1 + c3)*(-c2 + c3)*(c3 - g)**2) + b3/((-c0 + c3)*(-c1 + c3)*(-c2 + c3)*(c3 - g)**3) + b4/((-c0 + c3)*(-c1 + c3)*(-c2 + c3)*(c3 - g)**4))*exp(c3*t) + (b1/((c0 - c1)*(c0 - c2)*(c0 - c3)*(c0 - g)) + b2/((c0 - c1)*(c0 - c2)*(c0 - c3)*(c0 - g)**2) + b3/((c0 - c1)*(c0 - c2)*(c0 - c3)*(c0 - g)**3) + b4/((c0 - c1)*(c0 - c2)*(c0 - c3)*(c0 - g)**4))*exp(c0*t) + (b4*t**3/(6*(-c0 + g)*(-c1 + g)*(-c2 + g)*(-c3 + g)) + t**2*(b3/(2*(-c0 + g)*(-c1 + g)*(-c2 + g)*(-c3 + g)) + b4/(2*(-c0 + g)*(-c1 + g)*(-c2 + g)*(-c3 + g)*

In [ ]:
p[3][0][0]


In [ ]:
jetGF